<a href="https://colab.research.google.com/github/skywalker0803r/Wastewater-Biological-Treatment/blob/main/%E8%A9%95%E4%BC%B0critic%E6%A8%A1%E5%9E%8B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_percentage_error

# 載入模型權重
model = R2D2(obs_size, n_actions, hidden_size)
model.load_state_dict(torch.load("model_weights.pth"))

# 將模型設定為評估模式
model.eval()

# 評估模型
with torch.no_grad():
    # 將測試資料傳入模型，得到預測值
    y_pred = model.critic_forward(test_state, test_action, eval=True)

    # 計算 R2 score
    r2 = r2_score(test_value, y_pred)

    # 計算 MSE
    mse = mean_squared_error(test_value, y_pred)

    # 計算 MAPE
    mape = mean_absolute_percentage_error(test_value, y_pred)

print(f"R2 score: {r2:.4f}")
print(f"MSE: {mse:.4f}")
print(f"MAPE: {mape:.4f}")
